# Introduccion
En este notebook encontraréis el código para filtrar de los grandes ficheros de las reseñas de Amazon las empresas y marcas que nos imteresan.

Y además se alinean las reseñas con la meta información.

# Abrir los ficheros

In [1]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen

import random
import numpy as np
from tqdm import tqdm_notebook as tqdm
#from tqdm import tqdm.notebook.tqdm as tqdm
from collections import defaultdict

In [2]:
# load all metadata
#hau nire diskan daukat SAD karpetan 2020-2021

dfMergedfMeta = []#
   
#dfMergedfMeta= pd.read_csv('Cell_Phones/SamsungAppleXiaomiReviews_prueba.csv')
dfMergedfMeta= pd.read_csv('VG-Reviews5AndMetaElecNintSonyMic_v2.csv')




In [3]:
dfMergedfMeta[:5]

,Unnamed: 0,asin,vote,style,feature,similar_item,brand,also_buy,category,title,...,overall,reviewerID,reviewerName,unixReviewTime,reviewText,summary,City,State,Country,Postal Code
0,307,B00000DMAO,9999,-,[],NaN,Nintendo,"['B00002STI2', 'B00001QES1', 'B000021Y2Z', 'B0...","['Video Games', 'Retro Gaming & Microconsoles'...",1080 Snowboarding,...,4,A1ZPVW0WXNJ3YZ,"Alex ""47""",1144540800,I used to love this game; but now it's dated a...,Finally; a great snowboarding game.,Toledo,Parana,Brazil,NaN
1,308,B00000DMAO,9999,-,[],NaN,Nintendo,"['B00002STI2', 'B00001QES1', 'B000021Y2Z', 'B0...","['Video Games', 'Retro Gaming & Microconsoles'...",1080 Snowboarding,...,4,AQAMD65HGUGMO,cj the great,994982400,This game definitly fits the thought I had of ...,An Adequate N64 snowboarding game.,Uvinza,Kigoma,Tanzania,NaN
2,309,B00000DMAO,9999,-,[],NaN,Nintendo,"['B00002STI2', 'B00001QES1', 'B000021Y2Z', 'B0...","['Video Games', 'Retro Gaming & Microconsoles'...",1080 Snowboarding,...,3,A23MMS6ZRF4KJ4,"Craig Hightshoe ""TheCRH""",1289433600,I use to play this AGES ago and use to be pret...,Not quite what I remembered,Chicago,Illinois,United States,60610.0
3,310,B00000DMAO,9999,-,[],NaN,Nintendo,"['B00002STI2', 'B00001QES1', 'B000021Y2Z', 'B0...","['Video Games', 'Retro Gaming & Microconsoles'...",1080 Snowboarding,...,4,AODCE1AC7NOHW,"""cthoms""",983664000,"In comparison to other N64 games, this piece i...",1080 Degrees of Thumb Rotation,Suzhou,Anhui,China,NaN
4,311,B00000DMAO,9999,-,[],NaN,Nintendo,"['B00002STI2', 'B00001QES1', 'B000021Y2Z', 'B0...","['Video Games', 'Retro Gaming & Microconsoles'...",1080 Snowboarding,...,1,AWG2O9C42XW5G,"David ""Black Widow 2014""",1145923200,this is the stuppidiest game i've ever played....,1080 BORING!,Edinburgh,Scotland,United Kingdom,NaN


In [4]:
dfMergedfMeta['brand'].value_counts()

by\n    \n    Electronic Arts                        19949
by\n    \n    Nintendo                               16812
by\n    \n    Sony                                   10915
by\n    \n    Microsoft                               8810
Nintendo                                              4660
Electronic Arts                                       1780
Sony                                                  1697
Microsoft                                             1230
Sony Computer Entertainment                            438
by\n    \n    Sony Computer Entertainment America      106
Nintendo of America                                     98
Electronic Arts, Inc.                                   78
by\n    \n    Sony Computer Entertainment               74
Sony Entertainment                                      52
Microsoft Corporation                                   44
Super Nintendo Super Castlevania IV                     22
Electronc Arts                                          

In [5]:
dfMergedfMeta.describe()


,Unnamed: 0,vote,similar_item,overall,unixReviewTime,Postal Code
count,66796.000000,66796.0,0.0,66796.000000,6.679600e+04,13780.000000
mean,112238.725927,9999.0,NaN,4.159336,1.228237e+09,55260.133019
std,65760.692191,0.0,NaN,1.149041,1.335910e+08,32192.231346
min,307.000000,9999.0,NaN,1.000000,9.421920e+08,1040.000000
25%,55488.750000,9999.0,NaN,4.000000,1.103155e+09,23223.000000
50%,110435.500000,9999.0,NaN,5.000000,1.252541e+09,59503.000000
75%,170941.250000,9999.0,NaN,5.000000,1.357258e+09,90008.000000
max,229170.000000,9999.0,NaN,5.000000,1.405987e+09,99301.000000


In [6]:
dfMergedfMeta.keys()

Index(['Unnamed: 0', 'asin', 'vote', 'style', 'feature', 'similar_item',
       'brand', 'also_buy', 'category', 'title', 'main_cat', 'description',
       'reviewTime', 'overall', 'reviewerID', 'reviewerName', 'unixReviewTime',
       'reviewText', 'summary', 'City', 'State', 'Country', 'Postal Code'],
      dtype='object')

In [7]:
dfMergedfMeta['reviewText'].describe()

count                                                 66790
unique                                                52156
top       I'm a huge Zelda fan, and was looking for some...
freq                                                      4
Name: reviewText, dtype: object

- Vamos a ver si hay Nintendos

In [8]:
dfNintendo = dfMergedfMeta[dfMergedfMeta['brand'].str.contains('Nintendo',na=False)]

In [9]:
dfNintendo[dfNintendo.category.str.contains('Retro')]#por ejemplo los juegos retro

,Unnamed: 0,asin,vote,style,feature,similar_item,brand,also_buy,category,title,...,overall,reviewerID,reviewerName,unixReviewTime,reviewText,summary,City,State,Country,Postal Code
0,307,B00000DMAO,9999,-,[],NaN,Nintendo,"['B00002STI2', 'B00001QES1', 'B000021Y2Z', 'B0...","['Video Games', 'Retro Gaming & Microconsoles'...",1080 Snowboarding,...,4,A1ZPVW0WXNJ3YZ,"Alex ""47""",1144540800,I used to love this game; but now it's dated a...,Finally; a great snowboarding game.,Toledo,Parana,Brazil,NaN
1,308,B00000DMAO,9999,-,[],NaN,Nintendo,"['B00002STI2', 'B00001QES1', 'B000021Y2Z', 'B0...","['Video Games', 'Retro Gaming & Microconsoles'...",1080 Snowboarding,...,4,AQAMD65HGUGMO,cj the great,994982400,This game definitly fits the thought I had of ...,An Adequate N64 snowboarding game.,Uvinza,Kigoma,Tanzania,NaN
2,309,B00000DMAO,9999,-,[],NaN,Nintendo,"['B00002STI2', 'B00001QES1', 'B000021Y2Z', 'B0...","['Video Games', 'Retro Gaming & Microconsoles'...",1080 Snowboarding,...,3,A23MMS6ZRF4KJ4,"Craig Hightshoe ""TheCRH""",1289433600,I use to play this AGES ago and use to be pret...,Not quite what I remembered,Chicago,Illinois,United States,60610.0
3,310,B00000DMAO,9999,-,[],NaN,Nintendo,"['B00002STI2', 'B00001QES1', 'B000021Y2Z', 'B0...","['Video Games', 'Retro Gaming & Microconsoles'...",1080 Snowboarding,...,4,AODCE1AC7NOHW,"""cthoms""",983664000,"In comparison to other N64 games, this piece i...",1080 Degrees of Thumb Rotation,Suzhou,Anhui,China,NaN
4,311,B00000DMAO,9999,-,[],NaN,Nintendo,"['B00002STI2', 'B00001QES1', 'B000021Y2Z', 'B0...","['Video Games', 'Retro Gaming & Microconsoles'...",1080 Snowboarding,...,1,AWG2O9C42XW5G,"David ""Black Widow 2014""",1145923200,this is the stuppidiest game i've ever played....,1080 BORING!,Edinburgh,Scotland,United Kingdom,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52261,177058,B002Q6BFLE,9999,-,['Connect your GameBoy Advance to an electric ...,NaN,Nintendo,"['B000QJ1NS8', 'B01JOGH7SI', 'B01JYYWL7C', 'B0...","['Video Games', 'Retro Gaming & Microconsoles'...",Original Nintendo Gameboy Advance Sp AGS-002 A...,...,5,A1PK5EBCM2K3H4,Chris Jones,1397088000,This is exactly as it says. It is an original ...,Works Perfectly,New York City,New York,United States,10035.0
52262,177059,B002Q6BFLE,9999,-,['Connect your GameBoy Advance to an electric ...,NaN,Nintendo,"['B000QJ1NS8', 'B01JOGH7SI', 'B01JYYWL7C', 'B0...","['Video Games', 'Retro Gaming & Microconsoles'...",Original Nintendo Gameboy Advance Sp AGS-002 A...,...,5,A7CKXET12XIAV,Daniel,1370822400,Well if you have a Gameboy Advanced then you h...,Awesome,Houston,Texas,United States,77041.0
52263,177060,B002Q6BFLE,9999,-,['Connect your GameBoy Advance to an electric ...,NaN,Nintendo,"['B000QJ1NS8', 'B01JOGH7SI', 'B01JYYWL7C', 'B0...","['Video Games', 'Retro Gaming & Microconsoles'...",Original Nintendo Gameboy Advance Sp AGS-002 A...,...,5,AUN2L9ZRJCLIQ,digital.maverick,1335398400,This is the official Nintendo SP charger - no ...,What you are looking for,Córdoba,Veracruz,Mexico,NaN
52264,177061,B002Q6BFLE,9999,-,['Connect your GameBoy Advance to an electric ...,NaN,Nintendo,"['B000QJ1NS8', 'B01JOGH7SI', 'B01JYYWL7C', 'B0...","['Video Games', 'Retro Gaming & Microconsoles'...",Original Nintendo Gameboy Advance Sp AGS-002 A...,...,5,A12JARSR16QM21,Steve,1315440000,"If you need a way to charge your GBA SP, this ...",Most Basic GBA SP Charger,Katsina,Katsina,Nigeria,NaN


In [10]:
dfSony=dfMergedfMeta[dfMergedfMeta['brand'].str.contains('Sony|PlayStation|Electr',na=False)]

In [11]:
dfMicrosoft=dfMergedfMeta[dfMergedfMeta['brand'].str.contains('[M|m]icrosoft',na=False)]

In [12]:
dfNintendo['title'].value_counts()

Super Smash Bros Melee                            450
The Legend of Zelda: The Wind Waker               386
Metroid Prime                                     326
Super Mario Sunshine                              318
Luigi's Mansion - Gamecube                        258
                                                 ... 
Barbie Groom And Glam Pups - Nintendo Wii           5
Donkey Kong Jungle Beat with Bongos - Gamecube      5
Super Mario All-Stars / Super Mario World           5
Master of Illusion - Nintendo DS                    5
Crosswords PLUS - Nintendo 3DS                      5
Name: title, Length: 447, dtype: int64

In [13]:
dfSony['title'].value_counts()

Grand Theft Auto III                                                   510
God of War - PlayStation 2                                             372
Uncharted 2: Among Thieves - Playstation 3                             306
God of War III - Playstation 3                                         282
Killzone: Shadow Fall (PlayStation 4)                                  278
                                                                      ... 
Superman Returns - Xbox                                                  5
Everquest II: Sentinel's Fate - PC                                       5
Spongebob Squarepants Creature from the Krusty Krab - PlayStation 2      5
The Sims 2: Best of Business Collection - PC                             5
MVP Baseball 2005 - Gamecube                                             5
Name: title, Length: 1061, dtype: int64

In [14]:
dfMicrosoft['title'].value_counts()

Halo - Xbox                                                                    712
Halo 2 - Xbox                                                                  490
Halo 4 - Xbox 360 (Standard Game)                                              441
Xbox 360 Wireless Controller - Glossy Black                                    303
Halo Reach                                                                     268
                                                                              ... 
Xbox Live Arcade - Compilation Disc                                              5
Age of Empires III (PC Games)                                                    5
Xbox 360 Wireless Controller Blue                                                5
Microsoft Xbox 360 Special Edition Chrome Series Wireless Controller - Blue      5
State of Decay [Online Game Code]                                                5
Name: title, Length: 204, dtype: int64

## Seleccionar las reseñas positivas de Nintendo

In [15]:
dfNintendoPositives=dfNintendo[dfNintendo['overall']>3]

In [16]:
dfNintendoNegatives=dfNintendo[dfNintendo['overall']<=3]

## Generar los documentos (en este caso los textos de las reviews)

In [17]:
documents=dfNintendoNegatives[dfNintendoNegatives['reviewText'].notna()]['reviewText'].tolist()

In [18]:
documents[:10]

["I use to play this AGES ago and use to be pretty good. I started playing it again and I now found that the controls (at least for me) don't seem very forgiving. In order to do tricks you have to place the toggle in the exact correct place (at least pretty spot on). You also have to land in a specific position or else you crash (too many crashes makes it so you can't finish the race). The half-pipe mode is pretty decent though and can range from super easy to insanely hard. Overall a not so bad game for that time.",
 "this is the stuppidiest game i've ever played. I advise everyone not to play it or buy it. It really sucked.",
 'this game is the hardest game ive ever played, and that takes away the fun. also graphics very poor even for nintendo 64 cannot even recommend for people i dislike dont waste your time or money worst n64 game ever',
 'If you don\'t read anything else:If you find this game at a yard sale for $1.00 - $5.00 or online, then go ahead and get it, but in my opinion t

In [19]:


#!pygmentize helper_scripts/lda.py #if you want to see the code
#install scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import os
import pickle
import re
import pandas as pd
import numpy as np
import sys
#!{sys.executable} -m pip install plotly
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")




- https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html

In [ ]:
#@title Set topic modeling algorithm arguments

no_topics = 5 #@param {type:"integer"}

no_top_words = 30 #@param {type:"integer"}

no_top_documents = 10 #@param {type:"integer"}

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np

### Recordad lda_W almacena información sobre la distribución de los tópicos en los documentos

### lda_H almacena información sobre la distribución de los tópicos en las palabras

In [21]:
#@title Run LDA

def display_topics(H, W, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print("Topic %d:" % (topic_idx))
        print(''.join([' ' +feature_names[i] + ' ' + str(round(topic[i], 5)) #y esto también
                for i in topic.argsort()[:-no_top_words - 1:-1]]))
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        docProbArray=np.argsort(W[:,topic_idx])
        print(docProbArray)
        howMany=len(docProbArray);
        print("How Many");
        print(howMany);
        for doc_index in top_doc_indices:
            print(documents[doc_index])
            

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

# Run LDA
lda_model = LatentDirichletAllocation(n_components=no_topics, max_iter=100, learning_method='online', learning_offset=50.,random_state=0).fit(tf)
lda_W = lda_model.transform(tf)

lda_H=lda_model.components_ /lda_model.components_.sum(axis=1)[:, np.newaxis] 
print("LDA Topics")
display_topics(lda_H, lda_W, tf_feature_names, documents, no_top_words, no_top_documents)

LDA Topics
Topic 0:
 8217 0.00955 kombat 0.0064 mortal 0.0061 water 0.00602 mk 0.00578 platforms 0.0053 moves 0.00515 theme 0.00481 conker 0.00467 enemies 0.00402 galaxy 0.00402 stage 0.00364 wrestling 0.00357 way 0.00344 aonuma 0.00332 music 0.00323 camera 0.00316 fighting 0.00307 island 0.00301 bad 0.00299 yoshi 0.00288 rupees 0.00272 combat 0.00269 collecting 0.00252 end 0.0023 generic 0.00224 platform 0.00221 dungeons 0.00215 killer 0.00214 oh 0.00211
[ 109  105 2771 ...  929 1629 1635]
How Many
3295
First off be careful where you get your used controller from. I got mine from Finest Play and from what I can tell, these controllers were all rebuilt fron parts of other controllers that were broken in one form or another. I got one controller that has the orange spice plug at the end instead of the silver plug and the other one had a black plug from the black controller. They are fine functionally but they were in worse condition than avertised. Used very good means that all parts on

In [29]:
documents=dfNintendoPositives[dfNintendoPositives['reviewText'].notna()]['reviewText'].tolist()

In [30]:
documents

["I used to love this game; but now it's dated and now that I look back on it, it's not as fun now, either way, here's my review.Graphics: 8.5/10. This is a more realistic snowboarding game, so there are very realistic colors; unlike snowboarding kids or anything with very bright colors. 1080 features very realistic physics and is very sophisticated. Anyways, the graphics I remember loving mainly because of the powdery snow. I liked to edge in deep snow just because it was it was so fluffy. For that, this game got what it got in this category.Sound: 9/10. Whether you go through a foot of snow or right accross jagged ice, there's a different noise. There is a different sound for everything, which is strange because really for the most part in many games on the N64 there is the same or a very similar song for everything in the game.Control: 8/10. Very, very realistic and solid, but It's frustrating for that reason. it is really realistic so you really have to land FLAT to get land, which

In [31]:
#@title Run LDA

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

# Run LDA
lda_model = LatentDirichletAllocation(n_components=no_topics, max_iter=1000, learning_method='online', learning_offset=50.,random_state=0).fit(tf)
lda_W = lda_model.transform(tf)
lda_H = lda_model.components_

print("LDA Topics")
display_topics(lda_H, lda_W, tf_feature_names, documents, no_top_words, no_top_documents)

LDA Topics
Topic 0:
zelda link world time story series legend sword ocarina puzzles items adventure boss enemies princess dungeons wind final pikmin past dungeon waker quest twilight bosses magic man battle fantasy use
The Legend of Zelda: Ocarina of time is the best zelda game there is! You go from being a kid, to being an adult! Link must beat five temples as an adult, the forest temple, the fire temple, the water temple, the shadow temple, and the spirit temple. The bosses are awsome, and all the characters are cool!
This game is so much fun! You have your own boat and after the beginning level you are free to roam the sea as you please! You also have all the classic Zelda weapons: sword, shield, bow and arrows, boomarang, and bombs. In this game you also have a new tool called the wind waker, hence the name of the game, that lets you control the winds, such as redirceting the directing the wind blows, change day to night and back, and even summon hurricanes to teleport you all over